# Регулярные выражения


## Введение

В реальной жизни мы постоянно сталкиваемся с данными в текстовом виде. Часто нужно вычленить некоторые данные из файла и обработать их. Но как эффективнее это сделать? Можно писать длинную программу, манипулярующую строковым функциями, но есть и другое решение.

Регулярное выражение -- строка-шаблон, представляющая соой формальную запись для множества строк. (credits to @poldnev). То есть мы хотим описать нужные нам строки в некотором формате и потом их как-то вычленить и преобразовать. 

Далее в примерах мы будем использовать Python. Детали реализации в других языках могут несколько отличаться, но не очень сильно благодаря стандарту POSIX. Для начала импортируем библиотеку для работы с регулярными выражениями.

In [ ]:
!rm -rf regexp
!git clone https://github.com/Khabutdinov-Arslan/regexp.git
!cd regexp

Cloning into 'regexp'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [ ]:
import re

Полный список методов для работы с регулярными выражениями можно найти в документации, мы же в учебных целях ограничимся следующими:
* re.**search**(pattern, string) вовзвращает первое вхождение выражения в строку
* re.**findall**(pattern, string) возвращает список вхождений выражения в строку
* re.**sub**(pattern, replace, string) заменяет все вхождение выражения в строку на новое выражение, возвращает изменённую строку

### Пример: Лог

Пусть у нас есть выхлоп некоторой команды. Мы хотим вычленить из него адреса реплик.

In [ ]:
!cat regexp/fsck.txt

FSCK started by pd2020149 (auth:SIMPLE) from /93.175.29.107 at Thu Dec 31 16:49:53 UTC 2020

Block Id: blk_1073971176
Block belongs to: /data/wiki/en_articles_part/articles-part
No. of Expected Replica: 3
No. of live Replica: 3
No. of excess Replica: 0
No. of stale Replica: 0
No. of decommissioned Replica: 1
No. of decommissioning Replica: 0
No. of corrupted Replica: 0
Block replica on datanode/rack: mipt-node01.atp-fivt.org/default is HEALTHY
Block replica on datanode/rack: mipt-node09.atp-fivt.org/default is DECOMMISSIONED
Block replica on datanode/rack: mipt-node07.atp-fivt.org/default is HEALTHY
Block replica on datanode/rack: mipt-node03.atp-fivt.org/default is HEALTHY


Попробуем решить задачу в лоб известными нам инструментами -- строковыми функциями.

In [ ]:
log = ""
with open('regexp/fsck.txt', 'r') as log_file:
  log = log_file.read()
address = log.split('\n')[-2].split(' ')[-3].split('/')
print(address[0])

mipt-node03.atp-fivt.org


Какие есть проблемы?
* Читаемость кода
* Сложность модификации
* Сильно полагается на валидность данных

Решим ту же задачу с помощью регулярных выражений: 

In [ ]:
address = re.findall('(mipt-node\S+)\/', log)
print(address[0])

mipt-node01.atp-fivt.org


## Парсинг страницы

В этом разделе мы будем решать одну из наиболее распространённых задач: парсить веб-страницу. Для начала посмотрим на сам файл.

In [ ]:
html = ""
with open('regexp/example.html', 'r') as html_file:
  html = html_file.read()
print(html)

<!DOCTYPE html>
<html>
<head>
    <title>Example Domain</title>
    <meta charset="utf-8">
    <style type="text/css">
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    </style>    
</head>
<body class="vsc-initialized">
<div>
    <h1>Example Domain</h1>
    <h2>Some subheading containing example subdomain</h2>
    <p><a href="https://www.iana.org/domains/example">More information...</a></p>
    <p><span>C:\log.txt</span><span>File is located at C:\Users\Arslan\Documents\report.doc</span></p>
    <h2>Another subheading related to domain</h2>
    <h3>Broken heading</h2>
    <p>Another paragraph</p>
    <p>Some inspirational quoter</p><p>Same line paragraph</p>
    <table>
        <tr><th>Good</th><th>Price</th></tr>
        <tr><td>Coal</td><td>1$

## Базовые примитивы

Наверняка вы пользовались поиском файлов в своей любимой операционной системе. Когда вы хотели найти все скрипты, можно была написать что-то вроде ``*.py``. А если вам нужны были все номерные части "Шрека", то ``Shrek?.mp4``. При этом ``*`` означала произвольное количество вхождений какого-то символа, а ``?`` ровно одно. 

В синтаксисе регулярных выражений ``.`` означает произвольный символ.Также есть квантификаторы: ``*`` его повторение произвольное число раз. ``+`` используется, если повторение должно быть хотя бы одно, ``?`` означает 0 или 1 вхождение.

### Скобки

Для удобного разбиения на части выражения применяются скобки. Более того, мы можем обращаться к содержимому каждой пары скобок по номеру (нумеруются пары в порядке открытия). Содержимое скобок также называется группой захвата. 

Давайте начнём с простого: получим заголовок нашей страницы. Метод `findall` вернёт все вхождения группы в виде списка. Чтобы решить нашу задачу, просто создадим группу захвата, отвечающую содержимому внутри тега `<title>`.

In [ ]:
address = re.findall('<title>(.*)</title>', html)
print(address)

['Example Domain']


### Упражнение 1

Получите кодировку страницы.

In [1]:
charset = re.findall('your code here', html)
print(charset)

NameError: ignored

### Ленивые и жадные квантификаторы

Давайте теперь попробуем получить содержимое всех параграфов (они обозначаются тегом ``<p>``).

In [ ]:
paragraphs = re.findall('<p>(.*)</p>', html)
print(paragraphs)

['<a href="https://www.iana.org/domains/example">More information...</a>', '<span>C:\\log.txt</span><span>File is located at C:\\Users\\Arslan\\Documents\\report.doc</span>', 'Another paragraph', 'Some inspirational quoter</p><p>Same line paragraph', 'US zip-code is a just five digit number. Here are ranges for some states: Arizona 85001 thru 85055, Arkansas 72201 thru 72217 California 94203 thru 90213, Colorado 80201 thru 80239. ', 'Here are some IP ranges of Megafon:', 'For default dns server we recommend using 1.1.1.1 or 8.8.8.8', 'And some phone numbers from Tatarstan']


Всё ли распарсилось так, как мы хотели? Немного изменим выражение.

In [ ]:
paragraphs = re.findall('<p>(.*?)</p>', html)
print(paragraphs)

['<a href="https://www.iana.org/domains/example">More information...</a>', '<span>C:\\log.txt</span><span>File is located at C:\\Users\\Arslan\\Documents\\report.doc</span>', 'Another paragraph', 'Some inspirational quoter', 'Same line paragraph', 'US zip-code is a just five digit number. Here are ranges for some states: Arizona 85001 thru 85055, Arkansas 72201 thru 72217 California 94203 thru 90213, Colorado 80201 thru 80239. ', 'Here are some IP ranges of Megafon:', 'For default dns server we recommend using 1.1.1.1 or 8.8.8.8', 'And some phone numbers from Tatarstan']


В чём же было дело? По умолчанию ``.*`` пытается найти самое длинное совпадение при условии выполнения остальной части выражения. Часто, как например при работе с XML-подобными форматами, нам наоборот хочется найти наиболее короткое совпадение. Обычные квантификаторы, ищущие самое длинное совпадние, ``*`` и ``+`` называют жадными, у них есть ленивые версии ``*?`` и ``+?``. 

### Упражнение 2

Выведите все строки таблицы (содержимое тегов ``<tr>``).

In [ ]:
rows = re.findall('your code here', html)
print(rows)

[]


### Несколько групп захвата

Допустим, мы хотим распарсить таблицу глубже, а именно для каждой строки получить массив основ. Для этого мы можем просто использовать несколько групп захвата в одном выражении.

In [ ]:
rows = re.findall('<tr><td>(.*?)</td><td>(.*?)</td></tr>', html)
print(rows)

[('Coal', '1$'), ('Wool', '2.50$')]


Как видим, вполне ожидаемо вернулся двумерный массив. Что будет, если не использовать ленивые квантификаторы?

### Экранирование

Некоторые символы в синтаксисе зарезервированы как специальные: `.`, `?` и другие. Если мы хотим их использовать в прямом смысле внутри нашего выражения, надо писать ``\`` перед ними т. е. экранировать. Проблема: ``\`` используется для экранирования и в регулярных выражениях, и в питоновских строках. Чтобы это обойти, надо перед выражением писать ``r``. Поэтому если мы хотим получить пути к файлу внутри диска без буквы диска, надо всем этим воспользоваться.

In [ ]:
paths = re.findall(r'<span>C:\\(.*?)</span>', html) 
print(paths)

['log.txt']


Второй путь не нашёлся. Почему?

``<p><span>C:\log.txt</span><span>File is located at C:\Users\Arslan\Documents\report.doc</span></p>``



## Основные инструменты

### Пропуск вхождения

Как мы хотим разобраться с проблемой? Нам бы помогло создание групп захвата, которые бы пропускали все символы внутри тега ``<span>``. Но при этом мы не хотим засорять массив совпадений. И у нас есть такая возможность. Достаточно написать ``?:`` в начале выражения в скобках.

In [ ]:
paths = re.findall(r'<span>(?:.*?)C:\\(.*?)</span>', html) 
print(paths)

['log.txt', 'Users\\Arslan\\Documents\\report.doc']


### Выбор одного из символов

Допустим мы хотим выбрать все заголовки. Для этого нам пригодятся квадратные скобки. Внутри них можно задать символ, с одним из которых должно случиться совпадение. ``^`` в начале квадратных скобок означает выбор одного из символов, в них не перечисленных. Есть стандартные диапазона ``a-z``, ``0-9`` и другие.

In [ ]:
headings = re.findall(r'<h[1-6]>(.*?)</h[1-6]>', html) 
print(headings)

['Example Domain', 'Some subheading containing example subdomain', 'Another subheading related to domain', 'Broken heading']


### Стандартные классы и дополнительные квантификаторы

Допустим мы хотим выбрать из текста все zip-коды (считаем, что это просто 5-значные числа). Для обозначения цифр есть стандартный класс символов ``\d``, для обозначения повторения от x до y раз -- квантификатор ``{x,y}``. Подробнее разные классы можно почитать в документации.

In [ ]:
texts = re.findall(r'\d{5}', html)
print(texts)

['38488', '85001', '85055', '72201', '72217', '94203', '90213', '80201', '80239']


### Упражнение 3

Выведите номера телефонов. Не забудьте о том, что часть из них содержит код города.

In [ ]:
phones = re.findall(r'your code here', html)
print(phones)

[]


### Альтернативы

Через ``|`` можно перечислять альтернативные части выражения. Понятно, что по возможности стоит использовать предыдущую конструкцию в силу её большей произодительности. Для примера выберем все фразы односящийся к доменам и заголовкам.

In [ ]:
texts = re.findall(r'((?:sub)?(?:heading|domain))', html) 
print(texts)

['subheading', 'subdomain', 'domain', 'subheading', 'domain', 'heading']


А теперь посмотрите аккуратно на файл и поймите, что мы распарсили невалидный заголовок. Обычно это хорошее решение, но что если мы хотим игнорировать такие случаи?

### Обращение к группам скобок

In [ ]:
headings = re.findall(r'<h([1-6])>(.*?)</h(\1)>', html)
print(headings)

[('1', 'Example Domain', '1'), ('2', 'Some subheading containing example subdomain', '2'), ('2', 'Another subheading related to domain', '2')]


По ``\i`` можно обращаться к i-ой паре скобок из самого выражения. Правда, теперь мы захватываем лишний текст, но что поделать. Ещё группы можно называть, если писать ``?P<name>`` в начале скобок.

In [ ]:
row = re.search('<tr><td>(?P<good>.*?)</td><td>(?P<price>.*?)</td></tr>', html).groupdict()
print(row)

{'good': 'Coal', 'price': '1$'}


### Упражнение 4

Обрамите все zip-коды в тег ``<i>`` и выведите изменённый html.

In [ ]:
new_html = re.sub(r'your code here', r'your code here', html)
print(new_html)

<!DOCTYPE html>
<html>
<head>
    <title>Example Domain</title>
    <meta charset="utf-8">
    <style type="text/css">
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    </style>    
</head>
<body class="vsc-initialized">
<div>
    <h1>Example Domain</h1>
    <h2>Some subheading containing example subdomain</h2>
    <p><a href="https://www.iana.org/domains/example">More information...</a></p>
    <p><span>C:\log.txt</span><span>File is located at C:\Users\Arslan\Documents\report.doc</span></p>
    <h2>Another subheading related to domain</h2>
    <h3>Broken heading</h2>
    <p>Another paragraph</p>
    <p>Some inspirational quoter</p><p>Same line paragraph</p>
    <table>
        <tr><th>Good</th><th>Price</th></tr>
        <tr><td>Coal</td><td>1$

## Выделение IP-адресов

Чтобы вы не думали, что всё так просто, давайте поговорим о такой популярной задаче, как выделение IP-адресов и рассмотрим несколько подходов к его решению.

### Упражнение 5

Напишите свою реализацию

In [ ]:
address = re.findall('your code here', html)
print(address)

[]


Посмотрите на вывод. Всё ли хорошо?

### Наивное решение

Считаем, что IP это просто последовательность 4 чисел, разделённых точкой.

In [ ]:
address = re.findall('(\d+\.\d+\.\d+.\d+)', html)
print(address)

['31.173.128.0', '31.173.128.255', '188.170.162.0', '188.170.162.255', '85.26.175.0', '85.26.175.255', '1111.0.0.3', '000.000.000.9999', '0.0.0.0', '255.255.255.255', '1.1.1.1', '8.8.8.8']


### Количество цифр

Вспоминаем, что цифр в октете не просто ненулевое количество,а от 1 до 3.

In [ ]:
address = re.findall('(\d{1,3}\.\d{1,3}\.\d{1,3}.\d{1,3})', html)
print(address)

['31.173.128.0', '31.173.128.255', '188.170.162.0', '188.170.162.255', '85.26.175.0', '85.26.175.255', '111.0.0.3', '000.000.000.999', '0.0.0.0', '255.255.255.255', '1.1.1.1', '8.8.8.8']


### Обрамление

С обоих сторон от не цифр должны находиться не цифры.

In [ ]:
address = re.findall('\D(\d{1,3}\.\d{1,3}\.\d{1,3}.\d{1,3})\D', html) 
print(address)

['31.173.128.0', '31.173.128.255', '188.170.162.0', '188.170.162.255', '85.26.175.0', '85.26.175.255', '000.000.000', '0.0.0.0', '255.255.255.255', '1.1.1.1', '8.8.8.8']


### Границы адресов

Теперь нужно учтём, что октеты могут быть от 0 до 255.

In [ ]:
ip_regex = """\D((?:[01]?\d\d?|2[0-4]\d|25[0-5])
              \.(?:[01]?\d\d?|2[0-4]\d|25[0-5])
              \.(?:[01]?\d\d?|2[0-4]\d|25[0-5])
              \.(?:[01]?\d\d?|2[0-4]\d|25[0-5]))\D"""
address = re.findall(ip_regex, html, re.X) 
print(address)

['31.173.128.0', '31.173.128.255', '188.170.162.0', '188.170.162.255', '85.26.175.0', '85.26.175.255', '0.0.0.0', '255.255.255.255', '1.1.1.1', '8.8.8.8']


Победа? На самом деле нет. Адреса 0.0.0.0 и 255.255.255.255 служебные, их учитывать не надо. Да и так выражение кажется уже крайне громоздким. Спасает разнесение выражение на несколько строк. Для его использования надо передать флаг ``re.X``.

**При конструировании регулярных выражений важно найти разумный баланс между сложностью выражения и полнотой его покрытия**

## Выводы

Регулярные выражения это мощный и красивый инструмент для работы со строками. Хотя основы крайне просты, на то, чтобы стать мастером в этом деле, могут уйти годы. К счастью, для большинства бытовых случаев хватает изложенных выше конструкций. Но если вы хотите углубить свои познания, крайне рекомендую вам прочитать книжку из списка ниже.

## Ссылки

**Mastering Regular Expressions, 3rd edition by Jeffrey Friedl** -- главная книга о регулярных выражениях для тех, кто хочет максимально погрузиться в тему.

[Regex tester](https://regex101.com)

[Regex crossword](https://regexcrossword.com/)

[Шпаргалка от MIT](http://web.mit.edu/hackl/www/lab/turkshop/slides/regex-cheatsheet.pdf)

[Документация по работе с регулярными выражениями в Python](https://docs.python.org/3/library/re.html)

[Подробнее про квантификаторы](https://www.rexegg.com/regex-quantifiers.html)

[@ihateacm](https://t.me/ihateacm) мой телеграм, пишите если нашли в этом ноутбуке опечатки или другие неточности